# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298359422696                   -0.85    5.0
  2   -8.300179959959       -2.74       -1.25    1.0
  3   -8.300446005366       -3.58       -1.89    3.8
  4   -8.300462248634       -4.79       -2.78    1.2
  5   -8.300464434703       -5.66       -3.08    5.8
  6   -8.300464532804       -7.01       -3.25    1.0
  7   -8.300464586565       -7.27       -3.39    8.9
  8   -8.300464616153       -7.53       -3.52    1.0
  9   -8.300464636941       -7.68       -3.71    1.0
 10   -8.300464641444       -8.35       -3.88    1.0
 11   -8.300464643786       -8.63       -4.22    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67397919239                   -0.70    6.0
  2   -16.67855330143       -2.34       -1.14    1.4
  3   -16.67922581552       -3.17       -1.87    4.4
  4   -16.67928081671       -4.26       -2.77    2.0
  5   -16.67928611824       -5.28       -3.18    6.5
  6   -16.67928621907       -7.00       -3.50    3.2
  7   -16.67928622322       -8.38       -3.98    1.0
  8   -16.67928622435       -8.95       -4.60    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32619774777                   -0.56    6.9
  2   -33.33270730723       -2.19       -1.00    1.1
  3   -33.33415857472       -2.84       -1.74    5.0
  4   -33.33429591577       -3.86       -2.66    3.0
  5   -33.33694014553       -2.58       -2.51    9.4
  6   -33.33694115119       -6.00       -2.56    3.9
  7   -33.33694380577       -5.58       -3.10    2.5
  8   -33.33694391422       -6.96       -3.77    5.0
  9   -33.33694392654       -7.91       -4.09    4.8


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298543282343                   -0.85    5.2
  2   -8.300290541510       -2.76       -1.59    1.5
  3   -8.300459182530       -3.77       -2.64    4.8
  4   -8.300428456633   +   -4.51       -2.48    9.6
  5   -8.300464317079       -4.45       -3.53    2.2
  6   -8.300464587249       -6.57       -3.92    3.5
  7   -8.300464643023       -7.25       -4.77    2.2


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32573641449                   -0.56    6.9
  2   -33.32872733074       -2.52       -1.28    1.2
  3   -25.66082969682   +    0.88       -0.63    6.9
  4   -33.22821720891        0.88       -1.53    7.0
  5   -33.31699699614       -1.05       -1.58    3.0
  6   -33.33177996824       -1.83       -1.93    2.8
  7   -33.30334472302   +   -1.55       -1.79    3.5
  8   -33.31940177082       -1.79       -1.93    4.8
  9   -33.33493275353       -1.81       -2.39    2.5
 10   -33.33675567131       -2.74       -2.72    3.1
 11   -33.33675449292   +   -5.93       -2.82    2.4
 12   -33.33687240741       -3.93       -3.03    1.8
 13   -33.33690968473       -4.43       -3.19    2.1
 14   -33.33694089839       -4.51       -3.67    3.0
 15   -33.33694364874       -5.56       -4.29    3.2


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.